In [29]:
import unittest
import docker
import pymysql
import logging
import time
import uuid
import cryptography

In [30]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# MySQL configuration
MYSQL_IMAGE = "mysql:8.0"
MYSQL_PORT = 3307  # Using non-default port to avoid conflicts
MYSQL_ROOT_PASSWORD = "root"
MYSQL_DATABASE = "test_db"
MYSQL_USER = "test_user"
MYSQL_PASSWORD = "test_password"

In [31]:
class TestMySQLInfrastructure(unittest.TestCase):
    def setUp(self):
        """Set up MySQL container before each test."""
        logger.info("Setting up MySQL container...")
        self.client = docker.from_env()

        # Start MySQL container
        self.container = self.client.containers.run(
            MYSQL_IMAGE,
            environment={
                'MYSQL_ROOT_PASSWORD': MYSQL_ROOT_PASSWORD,
                'MYSQL_DATABASE': MYSQL_DATABASE,
                'MYSQL_USER': MYSQL_USER,
                'MYSQL_PASSWORD': MYSQL_PASSWORD,
                'MYSQL_AUTHENTICATION_PLUGIN': 'mysql_native_password'
            },
            ports={'3306/tcp': ('127.0.0.1', MYSQL_PORT)},
            detach=True,
            remove=True,
            name=f"mysql_test_{uuid.uuid4().hex[:8]}"
        )
        logger.info("Started setup")

        self._wait_for_mysql()

    def _wait_for_mysql(self, max_attempts: int = 30, delay: int = 2):
        """Wait for MySQL to be ready to accept connections."""
        for attempt in range(max_attempts):
            try:
                connection = pymysql.connect(
                    host='127.0.0.1',
                    port=MYSQL_PORT,
                    user='root',
                    password=MYSQL_ROOT_PASSWORD,
                    database=MYSQL_DATABASE,
                    connect_timeout=5
                )
                connection.close()
                logger.info(f"MySQL ready after {attempt + 1} attempts")
                return
            except Exception as e:
                logger.info(f"Attempt {attempt + 1}: MySQL not ready yet ({e})")
                time.sleep(delay)
        raise Exception("MySQL container failed to become ready")

    def tearDown(self):
        """Clean up after each test."""
        logger.info("Cleaning up MySQL container...")
        if hasattr(self, 'container') and self.container:
            self.container.stop()
            self.container.remove()

    def test_mysql_connection(self):
        """Test basic MySQL connection."""
        try:
            connection = pymysql.connect(
                host='127.0.0.1',
                port=MYSQL_PORT,
                user=MYSQL_USER,
                password=MYSQL_PASSWORD,
                database=MYSQL_DATABASE
            )
            self.assertTrue(connection.open, "Connection should be open")
            connection.close()
        except Exception as e:
            self.fail(f"Failed to connect to MySQL: {e}")

In [32]:
sql = TestMySQLInfrastructure()

In [33]:
sql.setUp()

INFO:__main__:Setting up MySQL container...


APIError: 500 Server Error for http+docker://localhost/v1.51/containers/faa74788a9db4edabf8e3673c8601feb35281f9f3f88cc8a9c9216149301ebd5/start: Internal Server Error ("failed to set up container networking: driver failed programming external connectivity on endpoint mysql_test_3d35ad62 (4e837879336fcc5994723226afef354a57f736f7cfccc3e927e2a97e117540ff): Bind for 127.0.0.1:3307 failed: port is already allocated")